In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import random

In [4]:
train_attn_path = '/content/drive/MyDrive/Event_Extraction/files/train_attn.txt'
test_attn_path = '/content/drive/MyDrive/Event_Extraction/files/test_attn.txt'

train_attn_sp_path = '/content/drive/MyDrive/Event_Extraction/files/train_attn_sp.txt'
val_attn_sp_path = '/content/drive/MyDrive/Event_Extraction/files/val_attn_sp.txt'
test_attn_sp_path = '/content/drive/MyDrive/Event_Extraction/files/test_attn_sp.txt'

train_answer_keys_path = '/content/drive/MyDrive/Event_Extraction/files/train_answer_keys.txt'
val_answer_keys_path = '/content/drive/MyDrive/Event_Extraction/files/val_answer_keys.txt'
test_answer_keys_path = '/content/drive/MyDrive/Event_Extraction/files//test_answer_keys.txt'

In [5]:
def add_sent_number(file_in, file_out):
    print(file_in)
    print(file_out)
    
    f_in = open(file_in, 'r')
    lines = f_in.readlines()
    f_in.close()
    
    f_out = open(file_out, 'w')
    for i in range(len(lines)):
        num = str(int(i+1))
        ln = num + " " + lines[i]
        f_out.write(ln)
    f_out.close()
    
# Call
add_sent_number(train_attn_path, train_attn_sp_path)
add_sent_number(test_attn_path, test_attn_sp_path)

/content/drive/MyDrive/Event_Extraction/files/train_attn.txt
/content/drive/MyDrive/Event_Extraction/files/train_attn_sp.txt
/content/drive/MyDrive/Event_Extraction/files/test_attn.txt
/content/drive/MyDrive/Event_Extraction/files/test_attn_sp.txt


In [6]:
def get_val_sent_index(): 
    
    global train_attn_sp_path
    
    label_to_sent_num = {} 

    f_in = open(train_attn_sp_path, 'r')
    lines = f_in.readlines()
    f_in.close()

    for l in lines: 
        l = l.strip().split(" ")[:2]
        num = int(l[0])
        lab = str(l[1])

        if lab not in label_to_sent_num: 
            label_to_sent_num[lab] = [] 

        label_to_sent_num[lab].append(num)


    val_index = []

    for l in label_to_sent_num:
        sent_num = label_to_sent_num[l]
        num = int(len(sent_num) / 10)
        random.shuffle(sent_num)
        random.shuffle(sent_num)
        val_index += sent_num[:num]

    val_index = sorted(val_index)
    print("len(val_index)", len(val_index))
    print("val_index[:5]", val_index[:5])
    return val_index


# Call
val_index = get_val_sent_index()

len(val_index) 792
val_index[:5] [4, 5, 27, 42, 45]


In [7]:
def train_val_split(val_index):
    global train_attn_sp_path, val_attn_sp_path

    f_in = open(train_attn_sp_path, 'r')
    lines = f_in.readlines()
    f_in.close()

    f_train = open(train_attn_sp_path, 'w')
    f_val = open(val_attn_sp_path, 'w')

    for l in lines: 
        l = l.strip().split(" ")
        num = int(l[0])
        lab = str(l[1])
        
        if num in val_index: 
            f_val.write(" ".join(l) + "\n")
        else: 
            f_train.write(" ".join(l) + "\n")
    
    f_train.close()
    f_val.close()
    
    print("Train - Val - Split ")

# Call
train_val_split(val_index)

Train - Val - Split 


In [11]:
def train_val_total_check(train_attn_sp_path, val_attn_sp_path):

    def get_count(file_path): 
        print(file_path)

        label_to_sent_count = {} 

        f_in = open(file_path, 'r')
        lines = f_in.readlines()
        f_in.close()

        for l in lines: 
            l = l.strip().split(" ")[:2]
            num = int(l[0])
            lab = str(l[1])

            if lab not in label_to_sent_count: 
                label_to_sent_count[lab] = 0 

            label_to_sent_count[lab] += 1
            
        return label_to_sent_count
    
    train = get_count(train_attn_sp_path)
    print(train)
    val = get_count(val_attn_sp_path)
    print(val)
    
    c = 0
    for l in train: 
        c += train[l]
        if l in val:
            c += val[l]
    
    print(c)


# Call
train_val_total_check(train_attn_sp_path, val_attn_sp_path)

/content/drive/MyDrive/Event_Extraction/files/train_attn_sp.txt
{'Component-Whole(e2,e1)': 424, 'Other': 1269, 'Instrument-Agency(e2,e1)': 367, 'Cause-Effect(e2,e1)': 594, 'Entity-Destination(e1,e2)': 760, 'Content-Container(e1,e2)': 337, 'Member-Collection(e1,e2)': 71, 'Message-Topic(e1,e2)': 441, 'Product-Producer(e2,e1)': 355, 'Member-Collection(e2,e1)': 551, 'Entity-Origin(e1,e2)': 512, 'Cause-Effect(e1,e2)': 310, 'Component-Whole(e1,e2)': 423, 'Message-Topic(e2,e1)': 130, 'Product-Producer(e1,e2)': 291, 'Entity-Origin(e2,e1)': 134, 'Content-Container(e2,e1)': 150, 'Instrument-Agency(e1,e2)': 88, 'Entity-Destination(e2,e1)': 1}
/content/drive/MyDrive/Event_Extraction/files/val_attn_sp.txt
{'Other': 141, 'Member-Collection(e1,e2)': 7, 'Cause-Effect(e1,e2)': 34, 'Content-Container(e1,e2)': 37, 'Cause-Effect(e2,e1)': 65, 'Member-Collection(e2,e1)': 61, 'Entity-Origin(e1,e2)': 56, 'Component-Whole(e1,e2)': 47, 'Product-Producer(e2,e1)': 39, 'Message-Topic(e1,e2)': 49, 'Message-Topic(e2

In [9]:
def create_answer_keys(in_file, out_file): 
    
    f_in = open(in_file, 'r')
    lines = f_in.readlines()
    f_in.close()

    f_out = open(out_file, 'w')
    
    for i in range(0, len(lines)):
        l = lines[i].strip().split(" ")
        num = str(i+1)
        lab = str(l[1])
        f_out.write(num + "\t" + lab) 
        f_out.write("\n") 
    f_out.close()
    
    print(out_file + " " + "Created")

In [10]:
create_answer_keys(train_attn_sp_path, train_answer_keys_path)
create_answer_keys(val_attn_sp_path, val_answer_keys_path)
create_answer_keys(test_attn_sp_path, test_answer_keys_path)

/content/drive/MyDrive/Event_Extraction/files/train_answer_keys.txt Created
/content/drive/MyDrive/Event_Extraction/files/val_answer_keys.txt Created
/content/drive/MyDrive/Event_Extraction/files//test_answer_keys.txt Created
